 # Market Value Predictor 

## Getting Started

Import necessarry libraries and dependencies

In [1]:
import pandas as pd  # Library that allows data manipulation and analysis
import numpy as np  # Library for high-level mathematical functions and support for multi-dimensional arrays
import matplotlib.pyplot as plt  # Library for plotting
import seaborn as sns  # Additional Library for plotting

# To import the necessarry libraries and dependendencies required for the machine learning models and their respective training and evaluation.

from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder 
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.neighbors import NearestNeighbors
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import (
    RandomForestClassifier,
    GradientBoostingClassifier,
    ExtraTreesClassifier,
)
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.svm import SVC
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from lightgbm import LGBMRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from skorch import NeuralNetRegressor


# To import metrics to measure performance
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.metrics.pairwise import cosine_similarity

# Pandas display options
pd.set_option("display.max_rows", None)  # To better display the rows when there are too many
pd.set_option("display.max_columns", None)  # To better display the columns when there are too many

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

Reading the file

In [2]:
df = pd.read_csv("C:/Users/aleja/OneDrive/Desktop/Data Analytics Msc/Thesis/male_players.csv") #Loading the CSV file 

## Exploratory data analysis

### Checking general information about the dataset

In [ ]:
df.info(verbose=True,show_counts=True) # For a summarry of the columns available, the amount of nulls per column and the data type 

- This initial glimpse of the code allows us to make quick inferences about the dataset being handled. We are dealing with 111 initial columns in our dataset which already shows us that we are probably going to need to perform data cleaning to get rid of unnecesarry columns.

- The Non-Null Count also signals that we must perform data cleaning as the count value are not the same for all columns.

- Lastly, the Data type of each column tells us how we are going to have to handle each column at the moment of making operations and when considering the steps to handle the different types of variables.  

In [ ]:
#We run the following method to get a initial look at the data.
df.head()

This overview of the dataset allows us to understand our next steps for data preprocessing: 
1. Player positions contain multiple positions separated by a comma. We will have to reduce it to only one so that comparisons can be made properly by avoiding having a mix of multiple positions being recognized as their own category. (This step was completed in Excel using the LEFT and FIND functions, for the sake of including various data analysis tools; hence the player_positions_2 colum).
1. Some of the scores assigned to each player for each position in the field (ls,st,rs,lw, etc.) include an operation, i.e. 90+3, which was included to include potential based on certain conditions. This causes the data type to be an object when it would be needed for these colums to be an integer.  

### Data Preprocessing

#### General Data Cleaning

We define a function that will allow us to perform the operations found in some of player position rating columns.

In [3]:
#To calculate the effective overall rating for the player position rankings
# Define a function to handle both addition and subtraction
def calculate_operation(x):
    x = str(x)  # Convert to string to avoid the error
    if '+' in x:
        return sum(map(int, x.split('+')))
    elif '-' in x:
        parts = x.split('-')
        return int(parts[0]) - int(parts[1])
    else:
        return int(x)

# Apply the function to multiple columns
columns_to_process = ['ls', 'st', 'rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']          

# Apply the function to the specified columns and update the DataFrame
for col in columns_to_process:
    if col in df.columns:  # Check if the column exists in the DataFrame
        df[col] = df[col].apply(calculate_operation)

We invert the values of ordinal variables like league level so that the logic for all cardinal variable stays consistent; higher is better.

In [4]:
df['league_level'] = 6 - df['league_level']

#### Handling Duplicates

By checking the dataset we see that there is a 'player_id' column that holds a unique identifier for each player. We use the following lines of code to check on duplicates. 

In [ ]:
duplicates = df[df.duplicated(subset=['player_id' ], keep=False)] #To check if there are any duplicates for player id
duplicates = duplicates.sort_values(by=['player_id']) #To sort and show the duplicated together in case there are any
duplicates


#### Handling Missing Values

It is crucial to handle missing values as they will skew our results and ML algorithms dont function properly with them. We first analyze the missing values in each column

In [ ]:
null_counts = df.isnull().sum() #To count the sum of the null values in our dataset
null_counts

As we can see, there are columns that have a high percentage of missing values, upwards of 90% in some cases. We decided the best course of action is to get rid of those columns all together and also drop some columns that dont provide valuable information at a first glance.

In [5]:
#We drop columns that have high null value count and that dont provide much information
df = df.drop(columns=['nation_jersey_number','player_traits' , 'goalkeeping_speed',
                       'club_loaned_from', 'nation_team_id', 'nation_position',
                       'player_tags', 'fifa_update', 'nationality_id', 'real_face',
                       'league_id','club_team_id','club_joined_date','update_as_of',
                       'dob', 'player_positions', 'body_type'
                       ]) 


Since the market value is our target variable, it is in our best interest to drop those records that have a null value in this column. We also drop those players that don't have a club position as it is basic for the types of players that we want to analyze.  

In [6]:
#To drop the rows of players missing their market value 
df = df.dropna(subset=['value_eur'])

#To drop the rows of players missing information
df = df.dropna(subset=['club_position'])

Now that we have solved the null values that could be disposed off, we must analyze what procedure to take to solve the null values in the columns that will prove useful to us. We will use histograms to check on the distribution for each of the variables and analyze the best method to impute said columns.

In [ ]:
h = df.hist(figsize = (25,25))

Plotting the histograms gave us interesting insights of our data. 
- The distributions for 'value_eur' and 'wage_eur' present a high degree of skewness to the left, which might affect our data. We will need to apply some transformation to correct this.
- The columns 'pace','shooting','passing','dribbling','mentality_composure' all present a normal distribution, which means that null values can be imputed using the mean.
- The columns 'defending','physic','release_clause_eur' present skewness in their distributions, meaning that null values can be imputed using the median.
- Colums with categorical variables that present null values like 'league_level' can be imputed using the mode. 

In [7]:
#To impute the columns that have a normal distribution with the mean
df['pace'].fillna(df['pace'].mean(), inplace=True)
df['shooting'].fillna(df['shooting'].mean(), inplace=True)
df['passing'].fillna(df['passing'].mean(), inplace=True)
df['dribbling'].fillna(df['dribbling'].mean(), inplace=True)
df['mentality_composure'].fillna(df['mentality_composure'].mean(), inplace=True)

#To impute the colums that present skewness with the median
df['defending'].fillna(df['defending'].median(), inplace=True)
df['physic'].fillna(df['physic'].median(), inplace=True)
df['release_clause_eur'].fillna(df['release_clause_eur'].median(), inplace=True)

#To impute the values of a ordinal variable with the mode
df['league_level'] = df['league_level'].fillna(df['league_level'].mode()[0])


Displaying the sum of the null counts to verify that are not any left. ('player_face_url' still presents null values but that is a column that will not be used in the analysis and is only needed for the construction of the Tableau Dashboard)

In [ ]:
null_counts = df.isnull().sum() #To count the sum of the null values in our dataset
null_counts

#### Handling Outliers

By taking a look a the histograms for each variable we could realize that we would need to apply data transformations techniques that would help us reduce the imbalance in our dataset. The histograms already give us a hint at potential outliers, however we use box plots to confirm take a much more precise look.

In [ ]:
b = df.plot(kind='box', figsize=(25, 25), subplots=True, layout=(10, 9))

There are multiple conclusions that we can draw from examining the box plots
- Some of the variables present a lot of outliers but in some of them it actually makes sense, i.e. height can present outliers on both ends of the box as goalkeepers are usually tall and some regions of the world have shorter players; not necessarily being determinant of player value. This teaches us that we must focus our outlier removal efforts in variables where outliers dont make sense or could potentially affect our results. 
- Other variables like the goalkeeping score present many outliers as it is logical that field players have low scores in this regard.  

#### Feature Scaling and Encoding

We implement a log transformation for some of the columns in our dataset that we previously recognized that had heavy skewness to one side.

In [ ]:
df['value_eur'] = np.log1p(df['value_eur'])
df['wage_eur'] = np.log1p(df['wage_eur'])
df['release_clause_eur'] = np.log1p(df['release_clause_eur'])

Now that the data inmbalance has been handled, it is advisable to put most of the numeric variables on similar scales so that they are more easily comparable. 

In [ ]:
columns_to_scale = ['value_eur','wage_eur','release_clause_eur', 'overall','potential',
                    'height_cm','weight_kg','pace',
                    'shooting','passing','dribbling','defending','physic',
                    'attacking_crossing','attacking_finishing','attacking_heading_accuracy',
                    'attacking_short_passing','attacking_volleys','skill_dribbling','skill_curve',
                    'skill_fk_accuracy','skill_long_passing','skill_ball_control','movement_acceleration',
                    'movement_sprint_speed','movement_agility','movement_reactions','movement_balance',
                    'power_shot_power','power_jumping','power_stamina','power_strength','power_long_shots',
                    'mentality_aggression','mentality_interceptions','mentality_positioning',
                    'mentality_vision','mentality_penalties','mentality_composure','defending_marking_awareness',
                    'defending_standing_tackle','defending_sliding_tackle','goalkeeping_diving',
                    'goalkeeping_handling','goalkeeping_kicking','goalkeeping_positioning',
                    'goalkeeping_reflexes','ls', 'st', 'rs','lw','lf','cf','rf','rw','lam',
                    'cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb',
                    'lcb','cb','rcb','rb','gk']

scaler = RobustScaler()
scaled_data = scaler.fit_transform(df[columns_to_scale])
df[columns_to_scale] = pd.DataFrame(scaled_data, index=df.index, columns=columns_to_scale)


Categorical variables have to be encoded for a better performance with our ML models

In [11]:
#We apply a Hot Encoder to the categorical variables that are not ordinal
columns_to_encode = ['player_positions_2','preferred_foot','work_rate','body_type_2']
encoder = OneHotEncoder(sparse_output=False, drop=None) 
encoded_data = encoder.fit_transform(df[columns_to_encode])
encoded_columns = encoder.get_feature_names_out(columns_to_encode)
encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns, index=df.index)
df = pd.concat([df.drop(columns=columns_to_encode), encoded_df], axis=1)



In [ ]:
df.head()

#### Dimensionality Reduction

Since keeping the interpretability of our data is of high importance and we want to focus on the relationships between our covariables and our target variables, we decided that the best option for dimensionality reduction would be to use embedded feature selection methods. These methods are more efficient as they perform the feature selection as part of the model building process. 

## ML Algorithms

In [12]:
exclude_columns = ['player_id', 'player_url','short_name','long_name','player_face_url','club_name','league_name','club_position','nationality_name']  # We exclude columns that are not necessary in the analysis and that contain strings


In [13]:

# Define features and target
X = df.drop(columns=['value_eur'] + exclude_columns)  # Features
y = df['value_eur']  # Target

# Split data into train and test based on 'fifa_version'
X_train = X[df['fifa_version'] == 22]
X_test = X[df['fifa_version'] == 23]
y_train = y[df['fifa_version'] == 22]
y_test = y[df['fifa_version'] == 23]



### Elastic Net

In [14]:

# Define Elastic Net model
elastic_net = ElasticNet()

# Define hyperparameter grid for GridSearchCV
param_grid = {
    'alpha': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 1]  # Mixing parameter (0 = Ridge, 1 = Lasso)
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)

# Print results
print("Best Hyperparameters:", grid_search.best_params_)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAPE:", mape)
print("R2 Score:", r2)

Fitting 5 folds for each of 30 candidates, totalling 150 fits
Best Hyperparameters: {'alpha': 0.01, 'l1_ratio': 0.5}
MAE: 2283251.333605772
MSE: 23529832688316.625
RMSE: 4850755.888345302
MAPE: 401.5849049968213
R2 Score: 0.5922950123725974


### Decision Trees

In [ ]:
# Define Decision Tree model
decision_tree = DecisionTreeRegressor(random_state=42)

param_grid = {
    'max_depth': [None, 10, 20, 30, 50],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required at a leaf node
    'max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider for splits
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)

# Print results
print("Best Hyperparameters:", grid_search.best_params_)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAPE:", mape)
print("R2 Score:", r2)

### Random Forest

In [ ]:
# Define Random Forest model
random_forest = RandomForestRegressor(random_state=42)

# Define hyperparameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 150, 200],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required at a leaf node
    'max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider for splits
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)

# Print results
print("Best Hyperparameters:", grid_search.best_params_)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAPE:", mape)
print("R2 Score:", r2)

# To print which were the most important features selected by the algorithm
feature_importances = best_model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
print("Feature Importances:\n", importance_df.sort_values(by='Importance', ascending=False))


### Gradient Boosting Machine

In [15]:
# Define Gradient Boosting model
gradient_boosting = GradientBoostingRegressor(random_state=42)

# Define hyperparameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of boosting stages
    'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
    'max_depth': [3, 5, 7],  # Maximum depth of the individual trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required at a leaf node
    'max_features': ['auto', 'sqrt', 'log2']  # Number of features to consider for splits
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=gradient_boosting, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)

# Print results
print("Best Hyperparameters:", grid_search.best_params_)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAPE:", mape)
print("R2 Score:", r2)

# To check on the importance of each of the features
feature_importances = best_model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
print("Feature Importances:\n", importance_df.sort_values(by='Importance', ascending=False))


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}
MAE: 316264.60150542925
MSE: 829730676339.1951
RMSE: 910895.5353602272
MAPE: 24.51655404553121
R2 Score: 0.985623130448398
Feature Importances:
                             Feature  Importance
13               release_clause_eur    0.155642
2                         potential    0.136842
33               movement_reactions    0.121989
62                              lam    0.085333
58                               lf    0.072880
1                           overall    0.051657
63                              cam    0.050046
69                               rm    0.034277
59                               cf    0.034102
64                              ram    0.027315
60                               rf    0.024538
71                              ldm    0.023758
65           

### Light GBM

In [ ]:
# Define LightGBM model
lgbm_model = LGBMRegressor(random_state=42, verbose=-1)

# Define hyperparameter grid for GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],  # Number of boosting stages
    'learning_rate': [0.2, 0.3, 0.4],  # Learning rate
    'max_depth': [3, 5, 7],  # Maximum depth of the individual trees
    'num_leaves': [31, 50, 75],  # Maximum number of leaves in one tree
    'min_child_samples': [20, 50, 100],  # Minimum number of data needed in a child (leaf)
    'subsample': [0.8, 0.9, 1.0],  # Subsample ratio of the training instances
    'colsample_bytree': [0.8, 0.9, 1.0],  # Subsample ratio of columns when constructing each tree
    'reg_alpha': [0, 0.1, 0.5],  # L1 regularization term on weights
    'reg_lambda': [0, 0.1, 0.5]  # L2 regularization term on weights
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=lgbm_model, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=2, verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test)

# Evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test, y_pred)

# Print results
print("Best Hyperparameters:", grid_search.best_params_)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAPE:", mape)
print("R2 Score:", r2)

# Feature Importance (optional)
feature_importances = best_model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
print("Feature Importances:\n", importance_df.sort_values(by='Importance', ascending=False))

### Neural Network with L1/L2 Regularization

In [ ]:
X_py = df.drop(columns=['value_eur'] + exclude_columns).values  # Convert DataFrame to NumPy array
y_py = df['value_eur'].values  # Convert Series to NumPy array

# Split data into train and test based on 'fifa_version'
X_train_py = X_py[df['fifa_version'] == 22]
X_test_py = X_py[df['fifa_version'] == 23]
y_train_py = y_py[df['fifa_version'] == 22]
y_test_py = y_py[df['fifa_version'] == 23]

In [ ]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_py, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_py, dtype=torch.float32) 
X_test_tensor = torch.tensor(X_test_py, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test_py, dtype=torch.float32)


In [ ]:
# Define the neural network architecture
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, l1_value, l2_value):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size4, 1)
        self.l1_value = l1_value
        self.l2_value = l2_value

    def forward(self, x):
        x = torch.relu(self.fc1(x)) # to apply the ReLU function to the first hidden layer
        x = torch.relu(self.fc2(x)) # to apply the ReLU function to the second hidden layer
        x = torch.relu(self.fc3(x)) # to apply the ReLU function to the third hidden layer
        x = torch.relu(self.fc4(x)) # to apply the ReLU function to the fourth layer
        x = self.fc5(x) # output layer no activation function required
        return x

    def l1_l2_regularization(self):
        l1_loss = sum(param.abs().sum() for param in self.parameters())
        l2_loss = sum(param.pow(2.0).sum() for param in self.parameters())
        return self.l1_value * l1_loss + self.l2_value * l2_loss

# Wrap the PyTorch model in a scikit-learn compatible regressor
net = NeuralNetRegressor(
    module=NeuralNet,
    module__input_size=X_train_tensor.shape[1],
    module__hidden_size1=512,
    module__hidden_size2=256,
    module__hidden_size3=128,
    module__hidden_size4=64,
    criterion=nn.MSELoss,
    optimizer=optim.Adam,
    max_epochs=300,
    batch_size=256,
    verbose=0,
    device='cuda' if torch.cuda.is_available() else 'cpu'
)

# Define hyperparameter grid for GridSearchCV
param_grid = {
    'module__l1_value': [0.2, 0.01],  # L1 regularization strength
    'module__l2_value': [0.0, 0.15],  # L2 regularization strength
    'max_epochs': [400],  # Number of epochs
    'batch_size': [64,128]  # Batch size
}



# Perform GridSearchCV
grid_search = GridSearchCV(estimator=net, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)
grid_search.fit(X_train_tensor.numpy(), y_train_tensor.numpy().ravel())  # To flatten y 


# Best model from grid search
best_model = grid_search.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test_tensor.numpy())

# Evaluation metrics
mae = mean_absolute_error(y_test_py, y_pred)
mse = mean_squared_error(y_test_py, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_py, y_pred)

# Calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test_py, y_pred)

# Print results
print("Best Hyperparameters:", grid_search.best_params_)
print("MAE:", mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("MAPE:", mape)
print("R2 Score:", r2)

## Output Generation

Now that we ran various ML algorithms and compared results we can take the last steps in our data pipeline. We choose the results from the best performing algorithm (or the one of our preference) and we take the final steps so that data is interpretable and ready to use for Tableau. 

In [18]:
#The best approach is to reload the original dataset, perform the changes and adhere the predicted column
df2 = pd.read_csv("C:/Users/aleja/OneDrive/Desktop/Data Analytics Msc/Thesis/male_players.csv")


We repeat some of the data preprocessing steps so that the dataset is ready to be used with Tableau

In [ ]:
def calculate_operation(x):
    x = str(x)  
    if '+' in x:
        return sum(map(int, x.split('+')))
    elif '-' in x:
        parts = x.split('-')
        return int(parts[0]) - int(parts[1])
    else:
        return int(x)

# Apply the function to multiple columns
columns_to_process = ['ls', 'st', 'rs','lw','lf','cf','rf','rw','lam','cam','ram','lm','lcm','cm','rcm','rm','lwb','ldm','cdm','rdm','rwb','lb','lcb','cb','rcb','rb','gk']          

for col in columns_to_process:
    if col in df2.columns:  
        df2[col] = df2[col].apply(calculate_operation)

In [ ]:
df2 = df2.dropna(subset=['value_eur'])
df2 = df2.dropna(subset=['club_position'])

In [ ]:
df2['pace'].fillna(df2['pace'].mean(), inplace=True)
df2['shooting'].fillna(df2['shooting'].mean(), inplace=True)
df2['passing'].fillna(df2['passing'].mean(), inplace=True)
df2['dribbling'].fillna(df2['dribbling'].mean(), inplace=True)
df2['mentality_composure'].fillna(df2['mentality_composure'].mean(), inplace=True)

df2['defending'].fillna(df2['defending'].median(), inplace=True)
df2['physic'].fillna(df2['physic'].median(), inplace=True)
df2['release_clause_eur'].fillna(df2['release_clause_eur'].median(), inplace=True)

df2['league_level'] = df2['league_level'].fillna(df2['league_level'].mode()[0])

We reverse the application of the Robust Scaler so that our target variable is comparable to original values

In [16]:
# Initialize the scaler
scaler = RobustScaler()

columns_to_scale = ['value_eur']

# Fit and transform the data
scaled_data = scaler.fit_transform(df[columns_to_scale])

# Access the median and IQR values used during scaling
median = scaler.center_  # Median values for each column
IQR = scaler.scale_      # IQR values for each column

# Reverse the scaling
original_data = scaled_data * IQR + median

df['Market_Value_Rev'] = original_data  # Add the predicted market values

We now reverse the application of the log transformation

In [19]:
df2['Predicted_Market_Value'] = np.expm1(df['Market_Value_Rev'])


Finally, we aperform a calculation to show the difference between the predicted value and the actual value

In [20]:
df2['Difference'] = df2['Predicted_Market_Value'] - df2['value_eur']


We finish by exporting our results into the data file of our choosing

In [ ]:
# Export to CSV
df2.to_csv('Player_value_predictions.csv', index=False)